In [32]:
import os
import time
import torch
import torch.nn as nn
from torch.nn import init
import torch.nn.functional as F
import numpy as np
import pandas as pd
import sklearn


dir_all_data='data/train.tsv'

#超参数设置
BATCH_SIZE = 32
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
#从文件中读取数据
data_all = pd.read_csv(dir_all_data, sep='\t')
print(data_all.shape)    #(156060, 4)
print(data_all.keys())   #['PhraseId', 'SentenceId', 'Phrase', 'Sentiment']
print(data_all.head())

(156060, 4)
Index(['PhraseId', 'SentenceId', 'Phrase', 'Sentiment'], dtype='object')
   PhraseId  SentenceId                                             Phrase  \
0         1           1  A series of escapades demonstrating the adage ...   
1         2           1  A series of escapades demonstrating the adage ...   
2         3           1                                           A series   
3         4           1                                                  A   
4         5           1                                             series   

   Sentiment  
0          1  
1          2  
2          2  
3          2  
4          2  


In [5]:
#shuffle、划分验证集、测试集,并保存
idx = np.arange(data_all.shape[0])
seed = 0
np.random.seed(seed)
np.random.shuffle(idx)  

train_size = int(len(idx) * 0.6)
test_size = int(len(idx) * 0.8)

data_all.iloc[idx[:train_size], :].to_csv('data/task2_train.csv', index=False)
data_all.iloc[idx[train_size:test_size], :].to_csv("data/task2_test.csv", index=False)
data_all.iloc[idx[test_size:], :].to_csv("data/task2_dev.csv", index=False)

[     0      1      2 ... 156057 156058 156059]
[ 21923  18206  81802 ... 152315 117952  43567]


In [6]:
#使用Torchtext采用声明式方法加载数据
#参考https://blog.csdn.net/JWoswin/article/details/92821752
from torchtext import data
PAD_TOKEN = '<pad>'
TEXT = data.Field(sequential=True, batch_first=True, lower=True, pad_token=PAD_TOKEN)
LABEL = data.Field(sequential=False, batch_first=True, unk_token=None)

In [8]:
#读取数据
datafields = [("PhraseId", None), # 不需要的filed设置为None
              ("SentenceId", None),
              ('Phrase', TEXT),
              ('Sentiment', LABEL)]
train_data = data.TabularDataset(path='data/task2_train.csv', format='csv', fields=datafields)
dev_data = data.TabularDataset(path='data/task2_dev.csv', format='csv', fields=datafields)
test_data = data.TabularDataset(path='data/task2_test.csv', format='csv', fields=datafields)

In [15]:
#构建词典，字符映射到embedding
#TEXT.vocab.vectors 就是词向量
TEXT.build_vocab(train_data, vectors='glove.6B.50d', 
                 unk_init= lambda x:torch.nn.init.uniform_(x, a=-0.25, b=0.25))
LABEL.build_vocab(train_data)
#得到索引，PAD_TOKEN='<pad>'
PAD_INDEX = TEXT.vocab.stoi[PAD_TOKEN]
TEXT.vocab.vectors[PAD_INDEX] = 0.0

In [16]:
print(TEXT.vocab.itos[1510])
print(TEXT.vocab.stoi['bore'])
# 词向量矩阵: TEXT.vocab.vectors
print(TEXT.vocab.vectors.shape)
word_vec = TEXT.vocab.vectors[TEXT.vocab.stoi['bore']]
print(word_vec.shape)
print(word_vec)

succeeds
1486
torch.Size([16473, 50])
torch.Size([50])
tensor([ 0.7493,  0.7730,  0.5915, -0.3801,  0.4761,  1.3279,  0.3476,  0.0737,
        -0.0291, -0.2731, -0.3928, -0.1822, -0.0110, -0.3036, -0.5352, -0.4523,
        -0.8613, -0.0940, -0.3921, -0.3335, -0.6319, -0.2460,  0.3667, -0.9392,
         0.3502, -0.9397, -1.1096,  0.8062,  0.5669, -0.3130,  1.5001, -0.1960,
         0.3081,  0.1727,  0.5624,  0.2619,  0.4756, -0.5688, -0.5013,  0.1903,
         0.0685, -0.0869, -0.1641, -0.2432,  0.3557, -0.1629, -0.1993, -0.1561,
         0.3508, -0.9423])


In [17]:
word_vec = TEXT.vocab.vectors[TEXT.vocab.stoi['<pad>']]
print(word_vec.shape)
print(word_vec)

torch.Size([50])
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0.])


In [40]:
#构建迭代器
train_iterator = data.BucketIterator(train_data, batch_size=BATCH_SIZE, train=True, shuffle=True, device=DEVICE)
dev_iterator = data.Iterator(dev_data, batch_size=BATCH_SIZE, train=False, sort=False, device=DEVICE) #batch_size应该为len(dev_data) 
test_iterator = data.Iterator(test_data, batch_size=BATCH_SIZE, train=False, sort=False, device=DEVICE)# 在 test_iter , sort一定要设置成 False, 要不然会被 torchtext 搞乱样本顺序

In [24]:
embedding_choice = 'glove'   #  'static'    'non-static'
num_embeddings = len(TEXT.vocab)
embedding_dim = 50
dropout_p = 0.5
filters_num = 100
vocab_size = len(TEXT.vocab)
label_num = len(LABEL.vocab)
print(vocab_size, label_num)

16473 6


In [29]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.embedding_choice = embedding_choice
        if self.embedding_choice == 'rand':
            self.embedding = nn.Embedding(num_embeddings, embedding_dim)
        elif self.embedding_choice == 'glove':
            self.embedding = nn.Embedding(num_embeddings, embedding_dim, padding_idx=PAD_INDEX) \
                                .from_pretrained(TEXT.vocab.vectors, freeze=True)
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=filters_num,  #卷积产生的通道
                               kernel_size=(3, embedding_dim), padding=(2,0))
        self.conv2 = nn.Conv2d(in_channels=1, out_channels=filters_num,  #卷积产生的通道
                               kernel_size=(4, embedding_dim), padding=(3,0))
        self.conv3 = nn.Conv2d(in_channels=1, out_channels=filters_num,  #卷积产生的通道
                               kernel_size=(5, embedding_dim), padding=(4,0))
        self.dropout = nn.Dropout(dropout_p)
        self.fc = nn.Linear(filters_num * 3, label_num)
        
    def forward(self, x):      # (Batch_size, Length) 
        x = self.embedding(x).unsqueeze(1) #(Batch_size, 1, Length, Dimention) 
        x1 = F.relu(self.conv1(x)).squeeze(3) #(Batch_size, filters_num, length+padding) 
        x1 = F.max_pool1d(x1, x1.size(2)).squeeze(2)  #(Batch_size, filters_num) 
        x2 = F.relu(self.conv2(x)).squeeze(3)  
        x2 = F.max_pool1d(x2, x2.size(2)).squeeze(2)      
        x3 = F.relu(self.conv3(x)).squeeze(3)  
        x3 = F.max_pool1d(x3, x3.size(2)).squeeze(2)      
        x = torch.cat((x1, x2, x3), dim=1)  #(Batch_size, filters_num *3)
        x = self.dropout(x)      #(Batch_size, filters_num *3)
        out = self.fc(x)       #(Batch_size, label_num)
        return out

In [30]:
#构建模型
model = CNN()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)#创建优化器SGD
criterion = nn.CrossEntropyLoss()   #损失函数
model.to(DEVICE)

CNN(
  (embedding): Embedding(16473, 50)
  (conv1): Conv2d(1, 100, kernel_size=(3, 50), stride=(1, 1), padding=(2, 0))
  (conv2): Conv2d(1, 100, kernel_size=(4, 50), stride=(1, 1), padding=(3, 0))
  (conv3): Conv2d(1, 100, kernel_size=(5, 50), stride=(1, 1), padding=(4, 0))
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=300, out_features=6, bias=True)
)

In [43]:
#开始训练
epoch = 5
best_accuracy = 0.0
start_time = time.time()

for i in range(epoch):
    model.train()
    total_loss = 0.0
    accuracy = 0.0
    total_correct = 0.0
    total_data_num = len(train_iterator.dataset)
    steps = 0.0
    for batch in train_iterator:
        steps += 1
        optimizer.zero_grad() #  梯度缓存清零
        batch_text = batch.Phrase
        batch_label = batch.Sentiment
        out = model(batch_text)    #[batch_size, label_num]
        loss = criterion(out, batch_label)
        total_loss += loss.item() 
        loss.backward()
        optimizer.step()        
        correct = (torch.max(out, dim=1)[1] == batch_label).sum()
        total_correct += correct.item()
        if steps % 100 == 0:
            print("Epoch %d_%.3f%%: Training average Loss: %f" 
                  % (i, steps * train_iterator.batch_size * 100 / len(train_iterator.dataset), total_loss / steps))  
    #每个epoch都验证一下
    model.eval()
    total_loss = 0.0
    accuracy = 0.0
    total_correct = 0.0
    total_data_num = len(dev_iterator.dataset)
    steps = 0.0    
    for batch in dev_iterator:
        steps += 1
        batch_text = batch.Phrase
        batch_label = batch.Sentiment
        out = model(batch_text)
        loss = criterion(out, batch_label)
        total_loss += loss.item()
        correct = (torch.max(out, dim=1)[1] == batch_label).sum()
        total_correct += correct.item()
        print("Epoch %d :  Verification average Loss: %f, Verification accuracy: %f%%, Total Time:%f"
          %(i, total_loss / steps, total_correct * 100 / total_data_num, time.time() - start_time))  
    if best_accuracy < total_correct / total_data_num:
        best_accuracy = total_correct / total_data_num 
        torch.save(model, 'model_saved/epoch_%d_accuracy_%f' % (i, total_correct / total_data_num))
        print('Model is saved in model_saved/epoch_%d_accuracy_%f' % (i, total_correct / total_data_num))

Epoch 0_3.417%: Training average Loss: 0.974355
Epoch 0_6.835%: Training average Loss: 0.977503
Epoch 0_10.252%: Training average Loss: 0.979830
Epoch 0_13.670%: Training average Loss: 0.982405
Epoch 0_17.087%: Training average Loss: 0.981506
Epoch 0_20.505%: Training average Loss: 0.982513
Epoch 0_23.922%: Training average Loss: 0.982074
Epoch 0_27.340%: Training average Loss: 0.981900
Epoch 0_30.757%: Training average Loss: 0.983388
Epoch 0_34.175%: Training average Loss: 0.983917
Epoch 0_37.592%: Training average Loss: 0.981239
Epoch 0_41.009%: Training average Loss: 0.979884
Epoch 0_44.427%: Training average Loss: 0.979721
Epoch 0_47.844%: Training average Loss: 0.980966
Epoch 0_51.262%: Training average Loss: 0.980850
Epoch 0_54.679%: Training average Loss: 0.980258
Epoch 0_58.097%: Training average Loss: 0.978798
Epoch 0_61.514%: Training average Loss: 0.978131
Epoch 0_64.932%: Training average Loss: 0.978174
Epoch 0_68.349%: Training average Loss: 0.978164
Epoch 0_71.767%: Train

Epoch 0 :  Verification average Loss: 0.931929, Verification accuracy: 4.168135%, Total Time:31.867910
Epoch 0 :  Verification average Loss: 0.934746, Verification accuracy: 4.219396%, Total Time:31.881909
Epoch 0 :  Verification average Loss: 0.935875, Verification accuracy: 4.277064%, Total Time:31.886917
Epoch 0 :  Verification average Loss: 0.936823, Verification accuracy: 4.328325%, Total Time:31.891915
Epoch 0 :  Verification average Loss: 0.935684, Verification accuracy: 4.395604%, Total Time:31.906906
Epoch 0 :  Verification average Loss: 0.931574, Verification accuracy: 4.469292%, Total Time:31.911903
Epoch 0 :  Verification average Loss: 0.935929, Verification accuracy: 4.530164%, Total Time:31.916882
Epoch 0 :  Verification average Loss: 0.935902, Verification accuracy: 4.591036%, Total Time:31.920881
Epoch 0 :  Verification average Loss: 0.938733, Verification accuracy: 4.635889%, Total Time:31.935880
Epoch 0 :  Verification average Loss: 0.937779, Verification accuracy: 4.

Epoch 0 :  Verification average Loss: 0.940806, Verification accuracy: 9.105181%, Total Time:32.491553
Epoch 0 :  Verification average Loss: 0.939833, Verification accuracy: 9.175664%, Total Time:32.496562
Epoch 0 :  Verification average Loss: 0.939882, Verification accuracy: 9.226925%, Total Time:32.501548
Epoch 0 :  Verification average Loss: 0.940631, Verification accuracy: 9.278185%, Total Time:32.515545
Epoch 0 :  Verification average Loss: 0.939853, Verification accuracy: 9.335854%, Total Time:32.520538
Epoch 0 :  Verification average Loss: 0.939189, Verification accuracy: 9.403133%, Total Time:32.524536
Epoch 0 :  Verification average Loss: 0.940684, Verification accuracy: 9.464005%, Total Time:32.530531
Epoch 0 :  Verification average Loss: 0.939790, Verification accuracy: 9.537693%, Total Time:32.535530
Epoch 0 :  Verification average Loss: 0.939595, Verification accuracy: 9.608176%, Total Time:32.549521
Epoch 0 :  Verification average Loss: 0.940080, Verification accuracy: 9.

Epoch 0 :  Verification average Loss: 0.943580, Verification accuracy: 13.990965%, Total Time:33.062228
Epoch 0 :  Verification average Loss: 0.943598, Verification accuracy: 14.045430%, Total Time:33.067243
Epoch 0 :  Verification average Loss: 0.943362, Verification accuracy: 14.112709%, Total Time:33.081235
Epoch 0 :  Verification average Loss: 0.943226, Verification accuracy: 14.176785%, Total Time:33.095227
Epoch 0 :  Verification average Loss: 0.942976, Verification accuracy: 14.244065%, Total Time:33.108208
Epoch 0 :  Verification average Loss: 0.942057, Verification accuracy: 14.320956%, Total Time:33.113200
Epoch 0 :  Verification average Loss: 0.941685, Verification accuracy: 14.394643%, Total Time:33.127206
Epoch 0 :  Verification average Loss: 0.941609, Verification accuracy: 14.452312%, Total Time:33.141202
Epoch 0 :  Verification average Loss: 0.942300, Verification accuracy: 14.506776%, Total Time:33.145202
Epoch 0 :  Verification average Loss: 0.941683, Verification acc

Epoch 0 :  Verification average Loss: 0.939104, Verification accuracy: 18.918399%, Total Time:33.673878
Epoch 0 :  Verification average Loss: 0.938488, Verification accuracy: 18.992087%, Total Time:33.687871
Epoch 0 :  Verification average Loss: 0.938477, Verification accuracy: 19.052959%, Total Time:33.691885
Epoch 0 :  Verification average Loss: 0.939323, Verification accuracy: 19.094608%, Total Time:33.705876
Epoch 0 :  Verification average Loss: 0.938875, Verification accuracy: 19.168295%, Total Time:33.718868
Epoch 0 :  Verification average Loss: 0.939820, Verification accuracy: 19.213148%, Total Time:33.732845
Epoch 0 :  Verification average Loss: 0.939959, Verification accuracy: 19.270817%, Total Time:33.746850
Epoch 0 :  Verification average Loss: 0.940310, Verification accuracy: 19.334893%, Total Time:33.751835
Epoch 0 :  Verification average Loss: 0.940552, Verification accuracy: 19.395765%, Total Time:33.755850
Epoch 0 :  Verification average Loss: 0.941170, Verification acc

...

Epoch 3 :  Verification average Loss: 0.925032, Verification accuracy: 43.594015%, Total Time:155.326935
Epoch 3 :  Verification average Loss: 0.924746, Verification accuracy: 43.667703%, Total Time:155.341925
Epoch 3 :  Verification average Loss: 0.924923, Verification accuracy: 43.734982%, Total Time:155.347929
Epoch 3 :  Verification average Loss: 0.924722, Verification accuracy: 43.802262%, Total Time:155.352919
Epoch 3 :  Verification average Loss: 0.924781, Verification accuracy: 43.869542%, Total Time:155.357919
Epoch 3 :  Verification average Loss: 0.925199, Verification accuracy: 43.917598%, Total Time:155.371907
Epoch 3 :  Verification average Loss: 0.925161, Verification accuracy: 43.991286%, Total Time:155.377905
Epoch 3 :  Verification average Loss: 0.924861, Verification accuracy: 44.055362%, Total Time:155.381913
Epoch 3 :  Verification average Loss: 0.924780, Verification accuracy: 44.113030%, Total Time:155.395907
Epoch 3 :  Verification average Loss: 0.924939, Verific

Epoch 3 :  Verification average Loss: 0.923443, Verification accuracy: 48.604748%, Total Time:155.897681
Epoch 3 :  Verification average Loss: 0.923221, Verification accuracy: 48.681639%, Total Time:155.901679
Epoch 3 :  Verification average Loss: 0.923024, Verification accuracy: 48.758530%, Total Time:155.915671
Epoch 3 :  Verification average Loss: 0.922975, Verification accuracy: 48.822606%, Total Time:155.919670
Epoch 3 :  Verification average Loss: 0.923383, Verification accuracy: 48.854644%, Total Time:155.933661
Epoch 3 :  Verification average Loss: 0.923928, Verification accuracy: 48.896293%, Total Time:155.938658
Epoch 3 :  Verification average Loss: 0.924324, Verification accuracy: 48.960369%, Total Time:155.951664
Epoch 3 :  Verification average Loss: 0.924195, Verification accuracy: 49.030853%, Total Time:155.957661
Epoch 3 :  Verification average Loss: 0.924153, Verification accuracy: 49.094928%, Total Time:155.972657
Epoch 3 :  Verification average Loss: 0.924487, Verific

Epoch 3 :  Verification average Loss: 0.925639, Verification accuracy: 53.484125%, Total Time:156.509133
Epoch 3 :  Verification average Loss: 0.925548, Verification accuracy: 53.548201%, Total Time:156.514114
Epoch 3 :  Verification average Loss: 0.925442, Verification accuracy: 53.615481%, Total Time:156.528104
Epoch 3 :  Verification average Loss: 0.925346, Verification accuracy: 53.685964%, Total Time:156.532117
Epoch 3 :  Verification average Loss: 0.925255, Verification accuracy: 53.743632%, Total Time:156.538099
Epoch 3 :  Verification average Loss: 0.925360, Verification accuracy: 53.817320%, Total Time:156.543097
Epoch 3 :  Verification average Loss: 0.925610, Verification accuracy: 53.871784%, Total Time:156.557088
Epoch 3 :  Verification average Loss: 0.925708, Verification accuracy: 53.935860%, Total Time:156.572079
Epoch 3 :  Verification average Loss: 0.926101, Verification accuracy: 53.983917%, Total Time:156.577078
Epoch 3 :  Verification average Loss: 0.925954, Verific

Epoch 3 :  Verification average Loss: 0.926849, Verification accuracy: 58.312242%, Total Time:157.107774
Epoch 3 :  Verification average Loss: 0.926903, Verification accuracy: 58.369910%, Total Time:157.113783
Epoch 3 :  Verification average Loss: 0.926589, Verification accuracy: 58.443597%, Total Time:157.117781
Epoch 3 :  Verification average Loss: 0.926720, Verification accuracy: 58.514081%, Total Time:157.131773
Epoch 3 :  Verification average Loss: 0.926726, Verification accuracy: 58.574953%, Total Time:157.136758
Epoch 3 :  Verification average Loss: 0.926718, Verification accuracy: 58.642232%, Total Time:157.142755
Epoch 3 :  Verification average Loss: 0.926585, Verification accuracy: 58.709512%, Total Time:157.156758
Epoch 3 :  Verification average Loss: 0.926381, Verification accuracy: 58.773588%, Total Time:157.161743
Epoch 3 :  Verification average Loss: 0.926107, Verification accuracy: 58.853683%, Total Time:157.166749
Epoch 3 :  Verification average Loss: 0.925960, Verific

KeyboardInterrupt: 

In [45]:
#测试-重新读取文件
PATH = 'model_saved/epoch_2_accuracy_0.619902'
model = torch.load(PATH)
model.to(DEVICE)
total_loss = 0.0
accuracy = 0.0
total_correct = 0.0
total_data_num = len(test_iterator.dataset)
steps = 0.0    
start_time = time.time()
for batch in test_iterator:
    steps += 1
    batch_text = batch.Phrase
    batch_label = batch.Sentiment
    out = model(batch_text)
    loss = criterion(out, batch_label)
    total_loss += loss.item()
    correct = (torch.max(out, dim=1)[1] == batch_label).sum()
    total_correct += correct.item()
print("Test average Loss: %f, Test accuracy: %f，Total time: %f"
  %(total_loss/steps, total_correct/total_data_num, time.time()-start_time) ) 

Test average Loss: 0.919537, Test accuracy: 0.621183，Total time: 5.643947
